# Creating workflows with CWT.

This tutorial show how to create workflows for CWT.

You need to get your API key from [here](https://aims2.llnl.gov/user/profile) and possibly authenticated via my_proxy_client or OAuth2 if the files you're trying to use require it.

In [1]:
host = 'aims2.llnl.gov'
verify = True
compute_token = None

import cwt

client = cwt.WPSClient('https://{!s}/wps/'.format(host), compute_token=compute_token, verify=verify)
client

WPSClient(url='https://aims2.llnl.gov/wps/', log=False, log_file=None, verify=True, version=None, cert=None, headers={})

In [ ]:
# (122, 240, 480)
inputs = cwt.Variable('http://aims3.llnl.gov/thredds/dodsC/css03_data/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r10i1p1f1/Amon/tas/gn/v20180830/tas_Amon_GISS-E2-1-G_historical_r10i1p1f1_gn_190101-195012.nc', 'tas')

In [ ]:
# First we subset the input
proc_subset = client.process_by_name('CDAT.subset')

proc_subset.add_inputs(inputs)

proc_subset.set_domain(cwt.Domain(time=('1910', '1925'), lat=(0, 90)))

proc_subset

In [ ]:
# Then we chain this result to the max operator
proc_max = client.process_by_name('CDAT.max')

proc_max.add_inputs(proc_subset)

proc_max.add_parameters(axes=['lat'])

proc_max

In [ ]:
# Now we obtain a workflow operator
proc_workflow = client.process_by_name('CDAT.workflow')
proc_workflow

In [ ]:
# and use proc_max.
client.execute(proc_workflow, [proc_max])

proc_workflow.wait()

In [ ]:
import cdms2

proc_output = [x for x in proc_workflow.output if 'CDAT.max' in x.name][0]

f = cdms2.open(proc_output.uri)

tas = f[proc_output.var_name]

# (3, 480)
print(tas.shape)

In [ ]:
import vcs

v = vcs.init()

v.plot(tas[0])